In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn import preprocessing

In [2]:
data_a_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_A.csv", low_memory = False);
data_b_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_B.csv", low_memory = False);
data_c_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_C.csv", low_memory = False);
data_d_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_D.csv", low_memory = False);

data_a_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_A.csv", low_memory = False);
data_b_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_B.csv", low_memory = False);
data_c_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_C.csv", low_memory = False);
data_d_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_D.csv", low_memory = False);
data_e_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_E.csv", low_memory = False);
data_f_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_F.csv", low_memory = False);
data_g_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_G.csv", low_memory = False);
data_h_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_H.csv", low_memory = False);

In [74]:
data_2014 = pd.concat([data_a_2014, data_b_2014, data_c_2014, data_d_2014], ignore_index=True)

data_2015 = pd.concat([data_a_2015, data_b_2015, data_c_2015, data_d_2015, 
                       data_e_2015, data_f_2015, data_g_2015, data_h_2015], 
                       ignore_index=True)

In [84]:
data_2015.zip_code.unique()

array(['381xx', '644xx', '030xx', '125xx', '064xx', '376xx', '604xx',
       '761xx', '493xx', '190xx', '174xx', '605xx', '023xx', '128xx',
       '231xx', '350xx', '351xx', '377xx', '300xx', '606xx', '958xx',
       '214xx', '431xx', '752xx', '200xx', '973xx', '280xx', '577xx',
       '430xx', '330xx', '142xx', '076xx', '441xx', '967xx', '482xx',
       '100xx', '331xx', '670xx', '750xx', '220xx', '019xx', '447xx',
       '850xx', '460xx', '117xx', '212xx', '672xx', '546xx', '453xx',
       '472xx', '434xx', '215xx', '936xx', '774xx', '439xx', '787xx',
       '236xx', '710xx', '462xx', '550xx', '661xx', '440xx', '197xx',
       '114xx', '354xx', '299xx', '636xx', '880xx', '219xx', '432xx',
       '856xx', '297xx', '275xx', '016xx', '015xx', '134xx', '916xx',
       '346xx', '238xx', '993xx', '852xx', '436xx', '226xx', '660xx',
       '054xx', '160xx', '477xx', '290xx', '707xx', '945xx', '029xx',
       '112xx', '347xx', '085xx', '984xx', '293xx', '802xx', '322xx',
       '471xx', '210

In [75]:
def format_revol(val):
    percent = val[:-1]
    return float(percent)/100

def format_interest(val):
    percent = val[:-1]
    return float(percent)/100

def format_term(val):
    term = val[1:3]
    return float(term)/12

In [76]:
def initial_model(df, columns, columns_to_encode, columns_to_normalize):
    min_max_scaler = preprocessing.MinMaxScaler()
    new_df = df.copy()
    new_df = new_df[columns]
    new_df = pd.get_dummies(new_df, columns = columns_to_encode)
    new_df['amnt'] = new_df['funded_amnt']
    new_df[columns_to_normalize] = min_max_scaler.fit_transform(new_df[columns_to_normalize])
    return new_df

def clean_data(old_df):
    df = old_df.copy()
    columns_to_drop = ['id', 'member_id', 'emp_title']
    df = df.drop(columns_to_drop, axis = 1)
    
    cols = ['zip_code', 'funded_amnt', 'emp_length', 'home_ownership', 'int_rate', 'purpose', 'total_pymnt',
        'annual_inc', 'verification_status', 'dti', 'loan_status', 'revol_util', 'grade', 'term']
    cols_encode = ['emp_length', 'home_ownership', 'verification_status', 'grade', 'purpose']
    cols_normalize = ['funded_amnt', 'annual_inc', 'dti']
    
    df = initial_model(df, cols, cols_encode, cols_normalize)
    
    df['term'] = df['term'].map(format_term, na_action = 'ignore')
    df['int_rate'] = df['int_rate'].map(format_interest, na_action='ignore')
    df['revol_util'] = df['revol_util'].map(format_revol, na_action='ignore')
    df['zip_code'] = df['zip_code'].str.slice(0,3)
    
    df[df['revol_util'].isnull()] = np.mean(df['revol_util'])
    df[df['int_rate'].isnull()] = np.mean(df['int_rate'])
    df = df[df.loan_status != 'Current']
    
    df['paid'] = 1;
    df.loc[df.loan_status == 'Fully Paid', 'paid'] = 0;
    
    return df

In [77]:
df_2014 = clean_data(data_2014)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [78]:
df_2015 = clean_data(data_2015)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [79]:
df_2015 = df_2015.drop(['purpose_educational'], axis = 1)

In [80]:
df_2015.columns == df_2014.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [81]:
df_2014 = df_2014.dropna()
df_2015 = df_2015.dropna()

In [83]:
df_2015.zip_code.unique()

array(['644', '030', '125', '604', '761', '493', '174', '605', '023',
       '351', '377', '300', '958', '752', '973', '280', '577', '142',
       '967', '482', '100', '331', '220', '019', '447', '850', '117',
       '212', '672', '546', '472', '774', '236', '710', '462', '550',
       '661', '440', '197', '114', '354', '299', '880', '219', '856',
       '297', '275', '016', '015', '916', '993', '852', '436', '226',
       '660', '160', '029', '112', '441', '347', '085', '984', '293',
       '802', '210', '070', '430', '681', '917', '928', '750', '770',
       '327', '945', '349', '910', '986', '136', '463', '301', '207',
       '303', '913', '607', '322', '606', '941', '940', '020', '551',
       '725', '138', '334', '282', '805', '038', '926', '594', '778',
       '711', '346', '250', '113', '341', '366', '957', '483', '370',
       '027', '010', '208', '800', '481', '104', '376', '111', '763',
       '082', '956', '164', '294', '600', '088', '704', '894', '980',
       '922', '206',

In [66]:
df_2014.zip_code = df_2014.zip_code.astype(float)
# df_2014.dtypes
df_2014[df_2014.zip_code < 1]

,zip_code,funded_amnt,int_rate,total_pymnt,annual_inc,dti,loan_status,revol_util,term,emp_length_1 year,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,amnt,paid
1133,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
3445,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
4166,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
5656,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
7168,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
7278,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
8609,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
9125,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
10411,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1
11352,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,...,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,0.556854,1


In [60]:
df_2014.to_csv('cleaned_2014', index = False)
df_2015.to_csv('cleaned_2015', index = False)